<a href="https://colab.research.google.com/github/LewisLabUCSD/AD_secretory_pathway/blob/master/Support_score_calculation_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/LewisLabUCSD/AD_secretory_pathway.git
%cd AD_secretory_pathway/

Cloning into 'AD_secretory_pathway'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 37 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (37/37), done.
/content/AD_secretory_pathway


In [2]:
!nvidia-smi
import feather
import torch
# torch.set_num_threads(32)
import numpy as np
import pickle as pickle
import socket
device = 'cuda'

Wed Feb 24 22:57:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from multiprocessing import Pool
from functools import partial
import socket
import feather
import pickle
project_root_dir = './'
from graph_gradient import * 
from load_python_datasets import load_expr_mat

Download supplemental files

In [4]:
## download parsed interaction database, prepared from 10.1016/j.cels.2018.03.001
!wget https://ndownloader.figshare.com/files/26600330
!mv 26600330 databases/int.db.PCNet.G.p.zip
!unzip databases/int.db.PCNet.G.p.zip  -d databases

--2021-02-24 22:57:44--  https://ndownloader.figshare.com/files/26600330
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 54.77.157.195, 34.255.241.20, 34.241.53.95, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|54.77.157.195|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/26600330/int.db.PCNet.G.p.zip [following]
--2021-02-24 22:57:44--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/26600330/int.db.PCNet.G.p.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.93.123
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.93.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51081294 (49M) [application/zip]
Saving to: ‘26600330’

26600330            100%[===================>]  48.71M  16.1MB/s    in 3.0s    

2021-02-24 22:57:48 (16.1 MB/s) - ‘26600330’ saved [51081294/51081294]

Ar

To use singel cell expression dataset, set:
```
use_expr = 'SC_counts'
transformation = 'sigmoid'
```
To use bulk expression data from MSBB, set:
```
use_expr = 'ROSMAP_fpkm'
transformation = 'log_sigmoid'
```

Otherwise, please specify your own expression matrix
Note that the first column of the expression matrix must be named "geneSymbol". The rest of the columns should be uniquely named after the samples from which the gene expression profiles were quantified. These sample names will be used as identifies for the calculated support scores.

| geneSymbol      | sample1 | sample2| ...|
| ----------- | ----------- |---|---|
| gene1      | $$v_\text{gene1, sample1}$$ | $$v_\text{gene1, sample2}$$||
| gene2   | $$v_\text{gene2, Isample}$$        | $$v_\text{gene2, sample2}$$||
|⋮||||



In [5]:
use_expression = "bulk_RNA"  #@param ['bulk_RNA', 'single_cell', 'custom']
if use_expression == 'custom':
    url_to_csv = '' 
    expr_mat= load_expr_mat(use_expr= url_to_csv, project_root_dir = project_root_dir, 
                            transformation='sigmoid')
elif use_expression == 'single_cell':
    ## download single cell expression dataset, prepared from 10.1038/s41586-019-1195-2
    !wget https://ndownloader.figshare.com/files/26599727
    !mv 26599727 databases/AD_sc_counts.feather.gz
    !gunzip databases/AD_sc_counts.feather.gz databases/AD_sc_counts.feather    
    expr_mat= load_expr_mat(use_expr='AD_sc_counts.feather', project_root_dir = project_root_dir + 'databases/', 
                        transformation='sigmoid')
elif use_expression == 'bulk_RNA':
    ## download bulk expression dataset, prepared from 10.1038/sdata.2018.185
    !wget https://ndownloader.figshare.com/files/26599763
    !mv 26599763 databases/AD_MSBB_scaleExp.feather.gz
    !gunzip databases/AD_MSBB_scaleExp.feather.gz databases/AD_MSBB_scaleExp.feather    
    expr_mat= load_expr_mat(use_expr='AD_MSBB_scaleExp.feather', project_root_dir = project_root_dir + 'databases/', 
                            transformation='sigmoid')

--2021-02-24 22:57:50--  https://ndownloader.figshare.com/files/26599763
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 54.171.25.147, 54.72.223.163, 108.128.236.202, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|54.171.25.147|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/26599763/AD_MSBB_scaleExp.feather.gz [following]
--2021-02-24 22:57:51--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/26599763/AD_MSBB_scaleExp.feather.gz
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.89.155
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.89.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136993642 (131M) [application/gzip]
Saving to: ‘26599763’

26599763            100%[===================>] 130.65M  28.5MB/s    in 5.3s    

2021-02-24 22:57:56 (24.7 MB/s) - ‘26599763’ saved [13

Load network (G) and secretory pathway components

In [6]:
G = pickle.load(open('%s/databases/int.db.PCNet.G.p' % project_root_dir, 'rb'))
secMs = feather.read_dataframe(
    '%s/databases/secM.components.feather' % project_root_dir).iloc[:, 0].to_list()
# secPs = feather.read_dataframe(
#     '%s/databases/secP.components.feather' % project_root_dir).iloc[:, 0].to_list()
sec_resident = feather.read_dataframe(
    '%s/databases/all.secretory.resident.genes.feather' % project_root_dir).iloc[:,
                0].to_list()
## Define secPs of interest
candidateSecPs = ['APP',
                  'ADAM10', 
                  'BACE1',
                  'PSEN1'
                  ]

Load expression data; 

In [7]:
patient_names = list(expr_mat.columns)
patient_ids = range(expr_mat.shape[1])
patient_name_dict = dict(zip(patient_ids, patient_names))
allGeneSymbols = list(expr_mat.index)

Start support score calculation

In [8]:
calculate_grad = False ## whether to calculate support score gradient
summarization = True ## Set to True to calculate one summary support score for each secreted protein as specified in candidateSecPs; set to False to calculate support component scores for each secreted protein.
calculate_stationary= False ## whether to calculate the stationary distribution of the expression-guided random walk
parallel = False ## When set to True, compute the support scores in parallel. Watch GPU memory usage.

def rwr_run(secP):
    try:
        if calculate_grad:
            other_context_genes = list(
                # set(G.nodes) - {secP} - set(secMs) - set(sec_resident)
                set(APP_pathway_genes + AD_risk_genes) - {secP} - set(secMs) - set(sec_resident)
                # set(secPs) - {secP} - set(secMs) - set(sec_resident)

            )  # list(set(candidateSecPs) - {secP} - set(secMs))
            a = RwrNode(secp=secP, G=G, secms=secMs, sec_resident=sec_resident,
                        other_context_genes=other_context_genes, device=device)
            mask_on = [x for x in a.G.nodes if x not in other_context_genes]
            vt_jacobian_nodes = 'secMs'
        else:
            other_context_genes = []
            a = RwrNode(secp=secP, G=G, secms=secMs, sec_resident=sec_resident,
                        other_context_genes=other_context_genes, device=device)
            mask_on = []
            vt_jacobian_nodes = None


    except Exception as e:
        # a = None
        # continuee
        return None

    res_dict = {}
    for patient_id in tqdm(patient_ids):
        res_dict.update({patient_name_dict[patient_id]:
                              a.p_arwr(mask_on=mask_on,
                                      secP_secM_expr=dict(zip(allGeneSymbols, expr_mat.iloc[:, patient_id])),
                                      summarization=summarization,  # 'context_weighted_secM'
                                      # n_prop=3,
                                      vt_jacobian_nodes=vt_jacobian_nodes, calculate_stationary=calculate_stationary
                                      )})

    return res_dict

# import tqdm
if parallel:
    with Pool(4) as p:
        res = list(tqdm(p.imap(rwr_run, candidateSecPs), total=len(candidateSecPs)))
else:
    res = []
    for secP in candidateSecPs:
        res.append(rwr_run(secP))



100%|██████████| 938/938 [01:58<00:00,  7.91it/s]


The output can be found under the ```AD_secretory_pathway/output``` folder

In [9]:
res_frame = pd.concat([pd.DataFrame.from_dict({(secP, i, j): res_dict[i][j]
                                                for i in res_dict.keys()
                                                for j in res_dict[i].keys()},
                                              orient='index') for secP, res_dict in zip(candidateSecPs, res) if res_dict is not None])
savepath='%s/output' %(project_root_dir)
from pathlib import Path
Path(savepath).mkdir(parents=True, exist_ok=True)

res_frame.to_csv('%s/RWR_secPs%s_%s.csv' % (savepath,
                                            'gradient' if calculate_grad else '',
                                            'summarized' if summarization else 'geneLevel'))
res_frame

context_weighted_all  ...  secM_avg
APP   S108B355.BM_10_739    summarized              0.758742  ...  0.224987
      S109B355.BM_10_791    summarized              0.751785  ...  0.226215
      S112B394.BM_10_621    summarized              0.737050  ...  0.224234
      S111B394.BM_10_569    summarized              0.739199  ...  0.224791
      S109B355.BM_10_788    summarized              0.749649  ...  0.225646
...                                                      ...  ...       ...
PSEN1 E009C189.hB_RNA_16695 summarized              0.729660  ...  0.245090
      L43C014.hB_RNA_4774   summarized              0.723928  ...  0.246442
      E009C189.hB_RNA_17075 summarized              0.727877  ...  0.245855
      K85C014.hB_RNA_8575   summarized              0.742472  ...  0.245435
      E009C189.hB_RNA_16805 summarized              0.731014  ...  0.246062

[3752 rows x 3 columns]